In [1]:
import os, pytensor
print("PYTENSOR_FLAGS =", os.environ.get("PYTENSOR_FLAGS"))
print("linker =", pytensor.config.linker)


PYTENSOR_FLAGS = None
linker = auto


In [1]:
# ============================================================
# PyMCでAR(1)（純Pythonモード / C/C++コンパイル不要）完全版
#   0) PowerShellで事前にこれで起動している前提：
#      $env:PYTENSOR_FLAGS="cxx=;linker=py;device=cpu"
#      python -m jupyter lab
# ============================================================

import numpy as np
import pandas as pd
import pymc as pm
import pytensor
import matplotlib.pyplot as plt
from io import StringIO

import os
print(os.environ.get("PYTENSOR_FLAGS"))
import pytensor
print(pytensor.config.linker)


# ------------------------------------------------------------
# 0) 純Pythonモード確認（最初に必ずチェック）
# ------------------------------------------------------------
print("PYTENSOR_FLAGS =", __import__("os").environ.get("PYTENSOR_FLAGS"))
print("pytensor version =", pytensor.__version__)
print("pymc version =", pm.__version__)
print("linker =", pytensor.config.linker)  # ここが "py" ならOK

# ------------------------------------------------------------
# 1) サンプルデータ（架空の株価 60本）
# ------------------------------------------------------------
csv_data = """date,close
2025-11-03,100.63
2025-11-04,100.49
2025-11-05,101.31
2025-11-06,103.21
2025-11-07,102.95
2025-11-10,102.72
2025-11-11,104.73
2025-11-12,105.73
2025-11-13,105.18
2025-11-14,105.90
2025-11-17,105.37
2025-11-18,104.85
2025-11-19,105.19
2025-11-20,102.91
2025-11-21,100.82
2025-11-24,100.17
2025-11-25,99.00
2025-11-26,99.41
2025-11-27,98.36
2025-11-28,96.72
2025-12-01,98.17
2025-12-02,97.93
2025-12-03,98.04
2025-12-04,96.39
2025-12-05,95.79
2025-12-08,95.93
2025-12-09,94.68
2025-12-10,95.15
2025-12-11,94.49
2025-12-12,94.19
2025-12-15,93.49
2025-12-16,95.12
2025-12-17,95.12
2025-12-18,93.86
2025-12-19,94.31
2025-12-22,92.95
2025-12-23,91.61
2025-12-24,91.93
2025-12-25,90.88
2025-12-26,91.18
2025-12-29,92.17
2025-12-30,92.42
2025-12-31,92.30
2026-01-01,91.28
2026-01-02,90.17
2026-01-05,89.61
2026-01-06,89.99
2026-01-07,90.40
2026-01-08,89.32
2026-01-09,89.73
2026-01-12,89.45
2026-01-13,89.65
2026-01-14,90.13
2026-01-15,90.17
2026-01-16,91.43
2026-01-19,90.81
2026-01-20,89.92
2026-01-21,89.62
2026-01-22,90.00
2026-01-23,91.09
"""

df = pd.read_csv(StringIO(csv_data))
y = df["close"].to_numpy(dtype=float)

# AR(1)のために (y_{t-1}, y_t) に分ける
y_prev = y[:-1]
y_next = y[1:]
y0 = y[0]

# ------------------------------------------------------------
# 2) PyMCモデル（AR(1) を尤度手書きで定義）
#    y_t ~ Normal(mu + phi*(y_{t-1}-mu), sigma)
# ------------------------------------------------------------
with pm.Model() as model:
    # 弱情報事前（適当に広い）
    mu = pm.Normal("mu", mu=float(np.mean(y)), sigma=50.0)
    phi = pm.Uniform("phi", lower=-0.99, upper=0.99)     # 安定性のため範囲制限
    sigma = pm.Exponential("sigma", lam=1.0)

    # 初期点（かなり緩い）
    pm.Normal("y0_like", mu=mu, sigma=10.0 * sigma, observed=y0)

    # 逐次尤度（ベクトルで一気に）
    pm.Normal(
        "y_like",
        mu=mu + phi * (y_prev - mu),
        sigma=sigma,
        observed=y_next
    )

    # サンプリング（純Python linker だと遅いので控えめ推奨）
    trace = pm.sample(
        draws=800,
        tune=800,
        chains=2,
        cores=1,
        target_accept=0.9,
        progressbar=True
    )

# ------------------------------------------------------------
# 3) 事後サンプルから未来10ステップ予測（シミュレーション）
# ------------------------------------------------------------
posterior = trace.posterior
mu_samps = posterior["mu"].stack(s=("chain", "draw")).values
phi_samps = posterior["phi"].stack(s=("chain", "draw")).values
sigma_samps = posterior["sigma"].stack(s=("chain", "draw")).values

def forecast_ar1(y_last, mu, phi, sigma, steps=10, rng=None):
    if rng is None:
        rng = np.random.default_rng()
    x = y_last
    out = np.empty(steps, dtype=float)
    for t in range(steps):
        x = rng.normal(mu + phi * (x - mu), sigma)
        out[t] = x
    return out

steps = 10
draws = min(1000, len(mu_samps))
rng = np.random.default_rng(0)

future = np.empty((draws, steps), dtype=float)
for i in range(draws):
    future[i] = forecast_ar1(y_last=y[-1], mu=mu_samps[-draws+i], phi=phi_samps[-draws+i], sigma=sigma_samps[-draws+i], steps=steps, rng=rng)

mean_pred = future.mean(axis=0)
p5 = np.percentile(future, 5, axis=0)
p95 = np.percentile(future, 95, axis=0)

# ------------------------------------------------------------
# 4) 可視化：観測系列 + 予測帯（5–95%）
# ------------------------------------------------------------
plt.figure(figsize=(11, 5))
plt.plot(np.arange(len(y)), y, label="observed")

x_future = np.arange(len(y), len(y) + steps)
plt.plot(x_future, mean_pred, label="mean forecast")
plt.fill_between(x_future, p5, p95, alpha=0.3, label="5–95% interval")

plt.title("PyMC Bayesian AR(1) (Pure Python PyTensor linker=py) + 10-step forecast")
plt.xlabel("time index")
plt.ylabel("price")
plt.legend()
plt.grid(True)
plt.show()


None
auto
PYTENSOR_FLAGS = None
pytensor version = 2.36.3
pymc version = 5.27.0
linker = auto


Initializing NUTS using jitter+adapt_diag...



You can find the C code in this temporary file: C:\Users\flare\AppData\Local\Temp\pytensor_compilation_error_uke7ce1m
library ine is not found.


CompileError: Compilation failed (return status=1):
"C:\cygwin64\bin\g++.EXE" -shared -g -Wno-c++11-narrowing -fno-exceptions -fno-unwind-tables -fno-asynchronous-unwind-tables -DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION -m64 -DMS_WIN64 -I"C:\Users\flare\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\_core\include" -I"C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\include" -I"C:\Users\flare\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pytensor\link\c\c_code" -L"C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\libs" -L"C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0" -o "C:\Users\flare\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\Local\PyTensor